In [ ]:
from astropy.time import Time
import astropy.units as u
import numpy as np
import matplotlib as mpl
from matplotlib import animation
import matplotlib.pyplot as plt
# %matplotlib inline

from twobody import KeplerElements, KeplerOrbit, TwoBodyKeplerElements

In [ ]:
t0 = Time.now()

## Two circular orbits

In [ ]:
def get_lim(x1, x2):
    min_ = np.min([np.min(x1[:2]), np.min(x2[:2])])
    max_ = np.max([np.max(x1[:2]), np.max(x2[:2])])

    lim = np.max([np.abs(min_), np.abs(max_)])
    ptp = 2*lim

    lim = [-lim - 0.04*ptp, 
           lim + 0.04*ptp]
    
    return lim

def get_orbit_animation(elem, t=None):
    orbit1 = KeplerOrbit(elem.primary)
    orbit2 = KeplerOrbit(elem.secondary)
    
    s1 = 300 * elem.m1.value ** 2
    s2 = 300 * elem.m2.value ** 2
    
    if t is None:
        dt = elem.P / 512
        t = elem.t0 + np.arange(0, elem.P.value+1e-5, dt.value) * elem.P.unit
        
    x1 = orbit1.reference_plane(t).cartesian.xyz.value
    x2 = orbit2.reference_plane(t).cartesian.xyz.value
    
    # Set up the figure to animate
    fig, ax = plt.subplots(1, 1, figsize=(6, 6))
    for k in ax.spines:
        ax.spines[k].set_visible(False)
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.set_facecolor('none')
    fig.set_facecolor('none')
    fig.tight_layout()
    
    min_ = np.min([np.min(x1[:2]), np.min(x2[:2])])
    max_ = np.max([np.max(x1[:2]), np.max(x2[:2])])
    
    lim = np.max([np.abs(min_), np.abs(max_)])
    ptp = 2*lim
    
    lim = [-lim - 0.04*ptp, 
           lim + 0.04*ptp]
    
    ax.plot(x1[0], x1[1], marker='', 
            lw=0.5, zorder=-100, alpha=1)
    ax.plot(x2[0], x2[1], marker='', 
            lw=0.5, zorder=-100, alpha=1)
    ax.scatter(0, 0, marker='+', s=100, color='#aaaaaa', zorder=-1000)
    
    data1 = ax.scatter(x1[0, 0], x1[1, 0], 
                       marker='o', s=s1, zorder=100)
    
    data2 = ax.scatter(x2[0, 0], x2[1, 0], 
                       marker='o', s=s2, zorder=100)
    
    ax.set_xlim(lim)
    ax.set_ylim(lim)
    
    def animate(i):
        data1.set_offsets(x1[:2, i:i+1].T)
        data2.set_offsets(x2[:2, i:i+1].T)
        return data1, data2
    
    interval = 25 * 128 / len(t)
    ani = animation.FuncAnimation(fig, animate, np.arange(0, len(t), 1),
                                  interval=interval, blit=True)
    
    return ani

In [ ]:
# elem = TwoBodyKeplerElements(P=10*u.day, 
#                              m1=1*u.Msun, m2=0.5*u.Msun,
#                              e=0, omega=0*u.deg, i=0*u.deg, Omega=0*u.deg, 
#                              M0=0*u.deg, t0=t0)

elem = TwoBodyKeplerElements(P=10*u.day, 
                             m1=1*u.Msun, m2=0.5*u.Msun,
                             e=0.8, omega=0*u.deg, i=0*u.deg, Omega=0*u.deg, 
                             M0=0*u.deg, t0=t0)

In [ ]:
anim = get_orbit_animation(elem)
# anim.save('../plots/derpy.mov', codec='png',
#           dpi=150, bitrate=-1, 
#           savefig_kwargs={'transparent': True, 'facecolor': 'none'})
anim.save('../plots/derpy.mp4',
          dpi=250, bitrate=-1)

# Orrery

In [ ]:
qs = np.concatenate(([0.01], np.linspace(0.1, 1, 4)))
es = np.linspace(0, 1, 5)
es[-1] = es[-1] - 0.01

eqs = np.stack(map(np.ravel, np.meshgrid(es, qs))).T

In [ ]:
P = 10 * u.day
dt = P / 512
t = elem.t0 + np.arange(0, P.value+1e-5, dt.value) * elem.P.unit

# Set up the figure to animate
fig, axes = plt.subplots(len(es), len(qs), figsize=(10, 10))

for ax in axes.flat:
    for k in ax.spines:
        ax.spines[k].set_visible(False)
    # ax.xaxis.set_visible(False)
    # ax.yaxis.set_visible(False)
    ax.xaxis.set_ticks([])
    ax.yaxis.set_ticks([])
    ax.set_facecolor('none')
    
for ax, e in zip(axes[0, :], es):
    ax.set_title('$e = {:.2f}$'.format(e), fontsize=18)
    
for ax, q in zip(axes[:, 0], qs):
    ax.set_ylabel('$q = {:.2f}$'.format(q), fontsize=18)
    
fig.set_facecolor('none')
fig.tight_layout()


datas = []
orbits = []
for (e, q), ax in zip(eqs, axes.flat):
    ax.scatter(0, 0, marker='+', s=70, 
               color='#aaaaaa', zorder=-1000)
    
    elem = TwoBodyKeplerElements(P=10*u.day, 
                                 m1=1*u.Msun, m2=1 * q*u.Msun,
                                 e=e, omega=0*u.deg, i=0*u.deg, Omega=0*u.deg, 
                                 M0=180*u.deg, t0=t0)
    
    orbit1 = KeplerOrbit(elem.primary)
    orbit2 = KeplerOrbit(elem.secondary)

    x1 = orbit1.reference_plane(t).cartesian.xyz.value
    x2 = orbit2.reference_plane(t).cartesian.xyz.value
    
    ax.plot(x1[0], x1[1], marker='', 
            lw=0.5, zorder=-100, alpha=1)
    ax.plot(x2[0], x2[1], marker='', 
            lw=0.5, zorder=-100, alpha=1)
    

    s1 = 75 * elem.m1.value
    s2 = max(75 * elem.m2.value, 10)
    
    data1 = ax.scatter(x1[0, 0], x1[1, 0], 
                       marker='o', s=s1, zorder=100)

    data2 = ax.scatter(x2[0, 0], x2[1, 0], 
                       marker='o', s=s2, zorder=100)
    
    lim = get_lim(x1, x2)
    ax.set_xlim(lim)
    ax.set_ylim(lim)
    
    datas.extend([data1, data2])
    orbits.extend([orbit1, orbit2])
    

def animate(i):
    for d, o in zip(datas, orbits):
        x = o.reference_plane(t[i:i+1]).cartesian.xyz.value
        d.set_offsets(x[:2].T)
    return datas

interval = 25 * 128 / len(t)
ani = animation.FuncAnimation(fig, animate, np.arange(0, len(t), 1),
                              interval=interval, blit=True)
ani.save('../plots/binary-star-gallery.mp4', 
         dpi=150, bitrate=-1)

In [ ]:
P = 10 * u.day
dt = P / 512
t = elem.t0 + np.arange(0, P.value+1e-5, dt.value) * elem.P.unit

# Set up the figure to animate
fig, axes = plt.subplots(len(es), len(qs), figsize=(10, 10))

for ax in axes.flat:
    for k in ax.spines:
        ax.spines[k].set_visible(False)
    # ax.xaxis.set_visible(False)
    # ax.yaxis.set_visible(False)
    ax.xaxis.set_ticks([])
    ax.yaxis.set_ticks([])
    ax.set_facecolor('none')
    
for ax, e in zip(axes[0, :], es):
    ax.set_title('$e = {:.2f}$'.format(e), fontsize=18)
    
for ax, q in zip(axes[:, 0], qs):
    ax.set_ylabel('$q = {:.2f}$'.format(q), fontsize=18)
    
fig.set_facecolor('none')
fig.tight_layout()


datas = []
orbits = []
for (e, q), ax in zip(eqs, axes.flat):
    ax.scatter(0, 0, marker='+', s=70, 
               color='#aaaaaa', zorder=-1000)
    
    elem = TwoBodyKeplerElements(P=10*u.day, 
                                 m1=1*u.Msun, m2=1 * q*u.Msun,
                                 e=e, omega=0*u.deg, i=90*u.deg, Omega=0*u.deg, 
                                 M0=180*u.deg, t0=t0)
    
    orbit1 = KeplerOrbit(elem.primary)
    orbit2 = KeplerOrbit(elem.secondary)

    rv1 = orbit1.radial_velocity(t).value
    rv2 = orbit2.radial_velocity(t).value
    
    ax.plot((t.jd-t0.jd), rv1, marker='', 
            lw=0.5, zorder=-100, alpha=1)
    ax.plot((t.jd-t0.jd), rv2, marker='', 
            lw=0.5, zorder=-100, alpha=1)
    

    s1 = 75 * elem.m1.value
    s2 = max(75 * elem.m2.value, 10)
    
    data1 = ax.scatter((t.jd-t0.jd)[0], rv1[0], 
                       marker='o', s=s1, zorder=100)

    data2 = ax.scatter((t.jd-t0.jd)[0], rv2[0], 
                       marker='o', s=s2, zorder=100)
    
    ax.set_xlim(0, P.value)
    
    lim = np.max(np.concatenate((rv1, rv2, np.abs(rv1), np.abs(rv2))))
    # ax.set_ylim(-lim, lim)
    ax.set_ylim(-250, 250)
    
    datas.extend([data1, data2])
    orbits.extend([orbit1, orbit2])
    

def animate(i):
    for d, o in zip(datas, orbits):
        x = o.radial_velocity(t[i:i+1]).value
        d.set_offsets(np.stack(((t[i:i+1].jd-t0.jd), x), axis=-1))
    
    return datas

interval = 25 * 128 / len(t)
ani = animation.FuncAnimation(fig, animate, np.arange(0, len(t), 1),
                              interval=interval, blit=True)
ani.save('../plots/binary-star-gallery-rvs.mp4', 
         dpi=150, bitrate=-1)